# pRealGARCH(S,p,q₁,q₂)

    GARCH equation

        loghₜ = ω(mod(t-1,m)+1) + ∑(r=1,..,p) βᵣ loghₜ₋ᵣ +  ∑(i=1,...,q₁) τ₁ᵢzₜ₋ᵢ  + ∑(j=1,...,q₂) τ₂ⱼ(zₜ₋ⱼ²-1) +  + γ′uₜ  for m = 1,...,S  
        S is the number of periodicity parameters. 

        rₜ = √hₜ zₜ,  zₜ ~ N(0,1)

    Measurement Equation

        log xₜ = ξ + log hₜ + δ(zₜ) + uₜ 

        where δ(z)=δ₁z + δ₂(z²-1)
 

# dRealGARCH(p,q)

    GARCH equation

        loghₜ = ω + ∑(r=1,..,p) βᵣ loghₜ₋ᵣ +  ∑(i=1,...,q) τ₁ᵢ̃zₜ₋ᵢ  + ∑(j=1,...,q) τ₂ⱼ(̃zₜ₋ⱼ²-1) +  + γ′̃uₜ    
        rₜ = √hₜ zₜ,  zₜ ~ N(0,1)

        ̃z and ̃u is defined by
        ̃z = z/sqrt(1+z^2*0.1) 
        ̃u = u/sqrt(1+u^2*0.1)

    Measurement Equation

        log xₜ = ξ + log hₜ + δ(zₜ) + uₜ 

        where δ(z)=δ₁z + δ₂(z²-1)
 


**Reference:**<br>
P.R. Hansen, Z Huang, H.H. Shek, 2012. Realized GARCH: A joint model for returns and realized measures of volatility. Journal of Applied Econometrics <br>
P.R. Hansen, Z Huang, 2015. Exponential GARCH modeling with realized measures of volatility. Journal of Business & Economic Statistics<br>
GD Banulescu, P.R. Hansen, Z Huang, M Matei, 2014. Volatility during the financial crisis through the lens of high frequency data: a realized GARCH approach<br>

In [1]:
#import package
using CSV, DataFrames
using Optim
using Plots
include("../src/ARCHModels.jl")
using .ARCHModels

In [ ]:
# load data
tickers = ["SPY","KO","AAPL","TSLA","JNJ","CVX"]
ticker = tickers[1] # Choose ticker 
filename = ticker*"_RMs.csv"#
readpath = dirname(pwd())*"\\src\\data\\"*filename
df = DataFrame(CSV.File(readpath,header = 1))

In [ ]:
# Price and Volatility plot
myplot = plot(df[:,1],df.close,title=ticker,color=:blue,legend=:topleft,label="Close Price",ylabel="Price(Dollar)",xlabel="Date",rightmargin = 1.5Plots.cm,yguidefont = font(:blue))
plt = twinx()
plot!(plt,df[:,1],df.RK,alpha=0.5,color=:red,legend=:top,label="RK",ylabel="σ²",yguidefont = font(:red))

In [ ]:
#split insample / out-of-sample
N₁ = 1000
# Close to Close return and Realized Kernel
c2c_r = diff(log.(df.close))
rm = df[2:end,"RK"]

rt = c2c_r
xt = rm
rt_N₁ = c2c_r[1:N₁]
xt_N₁ = rm[1:N₁]


In [ ]:

# In-sample estimation 

# egarch
#Refer to https://s-broda.github.io/ARCHModels.jl/stable/univariatetypehierarchy/ for EGARCH and TGARCH model specification. 

spec = EGARCH{1,1,1}(zeros(4))
am = UnivariateARCHModel(spec,rt_N₁)
fitted_egarch = fit(am)
fitted_coefs = fitted_egarch.spec.coefs
spec = EGARCH{1,1,1}(fitted_coefs)
am = UnivariateARCHModel(spec,rt)
ht_egarch_os = (volatilities(am).^2)[N₁+1:end]

# tgarch
spec = TGARCH{1,1,1}(zeros(4))
am = UnivariateARCHModel(spec,rt_N₁)
fitted_tgarch = fit(am)
fitted_coefs = fitted_tgarch.spec.coefs
spec = TGARCH{1,1,1}(fitted_coefs)
am = UnivariateARCHModel(spec,rt)
ht_tgarch_os = (volatilities(am).^2)[N₁+1:end]

# realgarch
spec = RealGARCH{1,1}(zeros(8)) # RealGARCH{p,q} = pRealGARCH{1,p,q,q} where q₁=q₂ 
am = UnivariateARCHXModel(spec,rt_N₁,xt_N₁)
fitted_realgarch = fit(am)
fitted_coefs = fitted_realgarch.spec.coefs
spec = RealGARCH{1,1}(fitted_coefs)
am = UnivariateARCHXModel(spec,rt,xt)
ht_realgarch_os = (volatilities(am).^2)[N₁+1:end]
println("complete")

# RealGARCH with fixed damper parameters (η=10)
spec = dRealGARCH{1,1}(zeros(8)) 
am = UnivariateARCHXModel(spec,rt_N₁,xt_N₁)
fitted_drealgarch = fit(am)
fitted_coefs = fitted_drealgarch.spec.coefs
spec = dRealGARCH{1,1}(fitted_coefs)
am = UnivariateARCHXModel(spec,rt,xt)
ht_drealgarch_os = (volatilities(am).^2)[N₁+1:end]
println("complete")



In [ ]:
fitted_egarch

In [ ]:
fitted_tgarch

In [ ]:
fitted_realgarch

In [ ]:
fitted_drealgarch

In [ ]:
# Out-of-sample forecast comparison
# qlike(σt,ht) = mean(σt./ht .- log.(σt./ht) .- 1) where σt and ht are realized volatility and volatility forecasting, repectively. 

σt2 = rt[N₁+1:end].^2 .+ 0.000001# squared return may have value 0. 


QLIKE = Dict()
QLIKE["RealGARCH"] = [qlike(σt2,ht_realgarch_os)]
QLIKE["dRealGARCH"] = [qlike(σt2,ht_drealgarch_os)]
QLIKE["EGARCH"] = [qlike(σt2,ht_egarch_os)]
QLIKE["TGARCH"] = [qlike(σt2,ht_tgarch_os)]


In [ ]:
#QLIKE table
QLIKE_table = DataFrame(QLIKE)[:,["TGARCH","EGARCH","RealGARCH","dRealGARCH"]]

In [ ]:
#Relative QLIKE table (QLIKE of RealGARCH normalized to 1)
relative_QLIKE_table = QLIKE_table ./ QLIKE_table[1,"RealGARCH"]